In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor
from xgboost  import XGBRegressor


In [9]:
# importing dataset
data = pd.read_csv(r'E:\machinelearningprojects\notebook\StudentsPerformance (1).csv')
data

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [14]:
# doing transformations using pipeline concept 
cat_cols = [cat_col for cat_col in data.columns if data[cat_col].dtype=='O']

In [15]:
cat_cols

['gender',
 'race/ethnicity',
 'parental level of education',
 'lunch',
 'test preparation course']

In [34]:
 cols = data.nunique()

for i in cat_cols:
   print( i,'--->',data[i].unique())

gender ---> ['female' 'male']
race/ethnicity ---> ['group B' 'group C' 'group A' 'group D' 'group E']
parental level of education ---> ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
lunch ---> ['standard' 'free/reduced']
test preparation course ---> ['none' 'completed']


In [25]:
data['gender'].unique()

array(['female', 'male'], dtype=object)

In [37]:
data.head(5)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [38]:
data['toal'] = data['math score']+data['reading score']+data['writing score']

In [41]:
cols = ['math score','reading score','writing score']
data.drop(columns = cols,axis=1,inplace=True)

In [42]:
data

,gender,race/ethnicity,parental level of education,lunch,test preparation course,toal
0,female,group B,bachelor's degree,standard,none,218
1,female,group C,some college,standard,completed,247
2,female,group B,master's degree,standard,none,278
3,male,group A,associate's degree,free/reduced,none,148
4,male,group C,some college,standard,none,229
...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,282
996,male,group C,high school,free/reduced,none,172
997,female,group C,high school,free/reduced,completed,195
998,female,group D,some college,standard,completed,223


In [43]:
features = data.iloc[:,:-1]

In [44]:
features.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course
0,female,group B,bachelor's degree,standard,none
1,female,group C,some college,standard,completed
2,female,group B,master's degree,standard,none
3,male,group A,associate's degree,free/reduced,none
4,male,group C,some college,standard,none


In [45]:
y = data.iloc[:,-1]
y

0      218
1      247
2      278
3      148
4      229
      ... 
995    282
996    172
997    195
998    223
999    249
Name: toal, Length: 1000, dtype: int64

In [ ]:
# columns transfer 
features.columns

'gender'

In [53]:
features.columns

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course'],
      dtype='object')

In [54]:
cat_features = ['gender', 'race/ethnicity', 'parental level of education', 'lunch','test preparation course']

In [58]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Specify the features to transform


# Define transformers for numeric and categorical features
numeric_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

# Define the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        
        ('one_hot_encoder', cat_transformer, cat_features)
    ]
)

# Assuming 'features' contains the dataset's feature columns
X = preprocessor.fit_transform(features)


In [72]:
X.shape

(1000, 17)

In [60]:
# train_test_spli
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [73]:
def evaluate(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    r2_score = r2_score(true,predicted)
    return mae,mse,r2_score

In [80]:
models = {
    "Linear regression" : LinearRegression(),
    "RandomForestRegressor":RandomForestRegressor(),
    "XGBRegressor":XGBRegressor(),
    'catBoost':CatBoostRegressor()
}
model_list = []
r2_list = []
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train,y_train)
     
# make predictions
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

print(list(models.keys()[i]))
model_list.append(list(models.keys()[i]))

print('model performence for training set')
print('mse:{:4f}'.format(model_train_mse))
print('mae:{:4f}'.format(model_train_mae))
print('r2_score:{:4f}'.format(model_train_r2_score))




Learning rate set to 0.039525
0:	learn: 42.0201165	total: 1.42ms	remaining: 1.42s
1:	learn: 41.7266061	total: 2.46ms	remaining: 1.23s
2:	learn: 41.4431627	total: 3.45ms	remaining: 1.15s
3:	learn: 41.1358134	total: 4.51ms	remaining: 1.12s
4:	learn: 40.8454220	total: 8.24ms	remaining: 1.64s
5:	learn: 40.5823993	total: 10.4ms	remaining: 1.73s
6:	learn: 40.3953154	total: 11.2ms	remaining: 1.58s
7:	learn: 40.1641258	total: 12.2ms	remaining: 1.51s
8:	learn: 39.9618975	total: 13.2ms	remaining: 1.45s
9:	learn: 39.7493594	total: 14ms	remaining: 1.38s
10:	learn: 39.5818975	total: 14.7ms	remaining: 1.32s
11:	learn: 39.3957588	total: 15.7ms	remaining: 1.29s
12:	learn: 39.2297203	total: 16.8ms	remaining: 1.27s
13:	learn: 39.0626895	total: 17.6ms	remaining: 1.24s
14:	learn: 38.9012621	total: 18.5ms	remaining: 1.21s
15:	learn: 38.7713676	total: 21.4ms	remaining: 1.32s
16:	learn: 38.6382699	total: 24.1ms	remaining: 1.39s
17:	learn: 38.4820354	total: 25.9ms	remaining: 1.41s
18:	learn: 38.3514874	total:

TypeError: 'dict_keys' object is not subscriptable

In [81]:
# Linear Regression
lr_model = LinearRegression()
lr_model.fit(x_train,y_train)
y_test_predict=lr_model.predict(x_test)

print(mean_squared_error(y_test_pred,y_test))
print(r2_score(y_test_pred,y_test))

2014.9332900626337
-1.6810554058841878
